In [ ]:
!pip install transformers -q
!pip install datasets -q
!pip install evaluate -q
!pip install jiwer -q
!pip install sacrebleu -q
!pip install pandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, Dataset, Audio, DatasetDict
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
import os
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
# print(tokenizer.tokenize("<bt>"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

['▁<', 'bt', '>']


In [ ]:
# "<bt>" in tokenizer.get_vocab()

False

In [ ]:
# "eng_Latn" in tokenizer.get_vocab()

True

In [ ]:
# print(tokenizer.tokenize("eng_Latn Hello_world"))

['eng_Latn', '▁Hello', '_', 'world']


# preprocessing Tatoeba

In [ ]:
train_dataset = load_dataset("ymoslem/Tatoeba-Translations",split="train", streaming=True)

In [ ]:
train_dataset

IterableDataset({
    features: ['id_src', 'lang_src', 'sentence_src', 'id_tgt', 'lang_tgt', 'sentence_tgt', 'lang_pair'],
    num_shards: 3
})

In [ ]:
english_sentences = (dataset["sentence_tgt"] for dataset in train_dataset if dataset["lang_tgt"]=="eng")

In [ ]:
# Example: print the first 5
lst_eng = []
limit = 300000
for i, sentence in tqdm(enumerate(english_sentences), total=limit, desc="Collecting sentences"):
    lst_eng.append(sentence)
    if i >= limit:
        break

In [ ]:
len(lst_eng)

300001

In [ ]:
eng_dataset = pd.DataFrame(lst_eng, columns=["text_en"])

In [ ]:
eng_dataset.to_csv("eng_300k_dataset.csv", index=False)

# translate

In [ ]:
!pip3 install ctranslate2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 25.6 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv("eng_300k_dataset.csv")

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['text_en'],
    num_rows: 300001
})

In [ ]:
dataset

Dataset({
    features: ['text_en'],
    num_rows: 300001
})

In [ ]:
mt_model_name = "kreasof-ai/nllb-200-distilled-600M-en2bem"
output_mt = "ct2-nllb-200-distilled-600M-finetuned"
!ct2-transformers-converter --model {mt_model_name} --quantization float16 --output_dir {output_mt}

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
config.json: 100% 906/906 [00:00<00:00, 4.77MB/s]
2025-04-11 10:14:53.924816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744366493.963119    6343 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744366493.978644    6343 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
model.safetensors: 100% 2.46G/2.46G [00:24<00:00, 102MB/s]
generation_config.json: 100% 154/154 [00:00<00:00, 1.15MB/s]
tokenizer_config.json: 100% 40.1k/40.1k [00:00<00:00, 7.5

In [ ]:
!wget https://s3.amazonaws.com/opennmt-models/nllb-200/flores200_sacrebleu_tokenizer_spm.model

--2025-04-11 10:16:14--  https://s3.amazonaws.com/opennmt-models/nllb-200/flores200_sacrebleu_tokenizer_spm.model
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.87.174, 54.231.170.192, 16.15.217.103, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.87.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4852054 (4.6M) [binary/octet-stream]
Saving to: ‘flores200_sacrebleu_tokenizer_spm.model’

flores200_sacrebleu 100%[===================>]   4.63M  8.37MB/s    in 0.6s    

2025-04-11 10:16:15 (8.37 MB/s) - ‘flores200_sacrebleu_tokenizer_spm.model’ saved [4852054/4852054]



In [ ]:
import os

directory = "/content/"

ct2_model_name = output_mt

ct_model_path = os.path.join(directory, ct2_model_name)
sp_model_path = os.path.join(directory, "flores200_sacrebleu_tokenizer_spm.model")

In [ ]:
import ctranslate2
import sentencepiece as spm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

sp = spm.SentencePieceProcessor()
sp.load(sp_model_path)

translator = ctranslate2.Translator(ct_model_path, device=device)

In [ ]:
# Translate
import numpy as np
# src_lang = "ben_Beng"
# tgt_lang = "eng_Latn"

src_lang = "eng_Latn"
tgt_lang = "bem_Latn"

beam_size = 5

source_sentences = dataset["text_en"]

source_sents = [sent.strip() for sent in source_sentences]
target_prefix = [[tgt_lang]] * len(source_sents)

# Subword the source sentences
source_sents_subworded = sp.encode_as_pieces(source_sents)
source_sents_subworded = [[src_lang] + sent + ["</s>"] for sent in source_sents_subworded]

# Translate the source sentences
translations = translator.translate_batch(source_sents_subworded,
                                          batch_type="tokens",
                                          max_batch_size=2024,
                                          beam_size=beam_size,
                                          return_scores=True,
                                          target_prefix=target_prefix)
# print("#"*10)
# print(translations)
lst_translations = [translation.hypotheses[0] for translation in translations]
# print(lst_translations)
lst_scores = [np.mean(translation.scores) for translation in translations]
norm_lst_scores = np.exp(lst_scores).tolist()
# print(lst_scores)
# print(norm_lst_scores)
# Desubword the target sentences
translations_desubword = sp.decode(lst_translations)
translations_desubword = [sent[len(tgt_lang):].strip() for sent in translations_desubword]


print(*translations_desubword[:10], sep="\n")

Tamuli ubusanso bwa mu mutwe iyo.
Tapali ubusanso bwa mu mutwe iyo.
Kwena twalemona kwati naciba fye ifyalinga ukuti bakalipe uyu shitata.
Umuntu umo uwiita ati Henry acilafwaya ukukumona.
Awe kanshi teti twise iyo lelo.
Tekulanda saana cita ifingi saana
Ukwesha kwena cintu icisuma saana.
awe ine nshakukabila iyo.
Ndemona kwati apa ndesubila ndeya kusukulu.
Awe kwena balakwipaya mwe!


In [ ]:
df_en_be = pd.DataFrame({"text_bem": translations_desubword, "text_en": source_sents, "score":norm_lst_scores})

In [ ]:
df_en_be["text_bem"]= df_en_be.text_bem.apply(lambda x: "<bt>"+" "+x)

In [ ]:
df_en_be.head()

,text_bem,text_en,score
0,<bt> Tamuli ubusanso bwa mu mutwe iyo.,There’s no psychological harm.,0.653848
1,<bt> Tapali ubusanso bwa mu mutwe iyo.,There is no psychological damage.,0.703359
2,<bt> Kwena twalemona kwati naciba fye ifyaling...,We thought that it was natural that he should ...,0.551431
3,<bt> Umuntu umo uwiita ati Henry acilafwaya uk...,Someone named Henry wanted to see you.,0.772525
4,<bt> Awe kanshi teti twise iyo lelo.,We can't come now.,0.578530


In [ ]:
df_en_be.to_csv("tatoeba_en_bem_300k.csv", index=False)

In [ ]:
len(df_en_be)

300001

In [ ]:
df_en_be.tail()

,text_bem,text_en,score
299996,<bt> Epakuti abana abakulu bangala ubwangalo b...,Lest the maturing children implement designs t...,0.532679
299997,<bt> Twaile uko aba bambi balonganine.,We went where the others had gathered.,0.680248
299998,<bt> Apa aambile ukulondolola ubucushi bwakwe ...,Here he began to recount his misfortune in det...,0.639731
299999,<bt> Takuli na umo uukwete insambu shakutampul...,Nobody has the right to take another's life.,0.681687
300000,<bt> Bushe finshi twingacetekela? Heathens!,What can we expect from them? Heathens!,0.744953


# Manually Checking The Translation based on score and filtering it


In [ ]:
df = pd.read_csv("tatoeba_en_bem_300k.csv")

In [ ]:
df.head()

,text_bem,text_en,score
0,<bt> Tamuli ubusanso bwa mu mutwe iyo.,There’s no psychological harm.,0.653848
1,<bt> Tapali ubusanso bwa mu mutwe iyo.,There is no psychological damage.,0.703359
2,<bt> Kwena twalemona kwati naciba fye ifyaling...,We thought that it was natural that he should ...,0.551431
3,<bt> Umuntu umo uwiita ati Henry acilafwaya uk...,Someone named Henry wanted to see you.,0.772525
4,<bt> Awe kanshi teti twise iyo lelo.,We can't come now.,0.578530


In [ ]:
# Manually checking

df[(df["score"] > 0.85) & (df["score"] < 0.9)].head()

,text_bem,text_en,score
199,<bt> mwati waliyapo ku Albania?,Have you ever been to Albania?,0.858022
243,<bt> Nomba mu ci Latin muli ifilembo twitatu a...,"Now, in Latin there are twenty-three letters: ...",0.869840
278,<bt> Alenwa ubwalwa.,He drank beer.,0.854406
532,<bt> Bushe naumona ubusuma bwa lucelo celo?,Do you see the beauty of this morning?,0.854302
533,"<bt> Muhammad peace be upon him, alifyalilwa m...","Muhammad, peace be upon him, was born in Mecca.",0.864302


In [ ]:
df_filtered = df[(df["score"] > 0.77)]

In [ ]:
df_filtered

,text_bem,text_en,score
3,<bt> Umuntu umo uwiita ati Henry acilafwaya uk...,Someone named Henry wanted to see you.,0.772525
29,<bt> Talelwala iyo.,He's not sick.,0.811748
54,<bt> Bushe bwangalo nshi watemwisha sana?,Which sport do you like the most?,0.835259
74,<bt> Bushe fyonse wacibelenga?,Did you read everything?,0.825797
103,<bt> Calisuminishiwa ukutwishika.,It's permissible to doubt.,0.779816
...,...,...,...
299919,<bt> Inshita yalusuba shitali ukucila inshita ...,The days are longer in summer than in winter.,0.789419
299930,<bt> Bushe cishinka ati na pali ba penguin ba ...,Is it true that even among the penguins of Ant...,0.797010
299948,<bt> Tom nasenda umwana wakwa Mary.,Tom delivered Mary's baby.,0.869004
299980,<bt> Bushe finshi ifyasangwako?,What have we found?,0.795092


In [ ]:
df_filtered.to_csv("tatoeba_en_bem_filtered_20k.csv", index=False)